In [2]:
# !pip install faiss-gpu==1.6.5
# !pip install tsnecuda==3.0.0 --no-deps
# !pip install mkl
# # need to install MKL: https://github.com/eddelbuettel/mkl4deb

In [5]:
import json
import days.w3d2.w3d2_tests as w3d2_tests
import transformers
import torch
from einops import rearrange, reduce, repeat

from sklearn.decomposition import PCA
from tsnecuda import TSNE
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

In [10]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
tokenizer._add_tokens(["[BEGIN]", "[END]"])
tokenizer.pad_token = "[END]"
tokenizer.eos_token = "[END]"
# 50258 is the pad_token_id
# 50257 is the BEGIN token id

with open("../w3d2/test_tokens_owt_subset.json") as f:
    test_tokens = torch.LongTensor(json.load(f))[:,:512]

sentence = (
"Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly " +
"normal, thank you very much. They were the last people you’d expect to be involved in anything " +
"strange or mysterious, because they just didn’t hold with such nonsense.\n" +
"Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy " +
"man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin " +
"and blonde and had nearly twice the usual amount of neck, which came in very useful as she " +
"spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys " +
"had a small son called Dudley and in their opinion there was no finer boy anywhere.\n" +
"The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was " +
"that somebody would discover it. They didn’t think they could bear it if anyone found out about " +
"the Potters. Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t met for several years; in fact, " +
"Mrs. Dursley pretended she didn’t have a sister, because her sister and her good-for-nothing " +
"husband were as unDursleyish as it was possible to be. The Dursleys shuddered to think what the " +
"neighbors would say if the Potters arrived in the street. The Dursleys knew that the Potters had a " +
"small son, too, but they had never even seen him. This boy was another good reason for keeping " +
"the Potters away; they didn’t want Dudley mixing with a child like that."
)
sentence_tokens = tokenizer(sentence).input_ids

In [7]:
# True -> use GPT
# False -> use 2-layer attention-only model
use_gpt = False

if use_gpt:
    model = transformers.GPT2Model.from_pretrained('gpt2')
    VOCAB_SIZE = 50257
    MAX_SEQ_LEN = 1024
    token_embed_fn = model.wte
    pos_embed_fn = model.wpe
else:
    model = w3d2_tests.get_minigpt("../w3d2/model.pt")
    VOCAB_SIZE = 50259
    MAX_SEQ_LEN = 512
    token_embed_fn = model.token_embedding
    pos_embed_fn = model.pos_embedding

In [12]:
with torch.no_grad():
    all_token_embeddings = token_embed_fn(torch.LongTensor(sentence_tokens))
all_token_embeddings.shape

torch.Size([359, 256])

In [13]:
with torch.no_grad():
    all_position_embeddings = pos_embed_fn(torch.arange(len(sentence_tokens)))
all_position_embeddings.shape

torch.Size([359, 256])

In [29]:
X = torch.concat([all_token_embeddings, all_position_embeddings], dim=0)

pca = PCA(n_components=2)
print(all_token_embeddings.shape)
projected = pca.fit_transform(X)
projected_transpose = rearrange(projected, "x y -> y x")

torch.Size([345, 256])


In [30]:
tsne_input = X
tsne = TSNE()
tsne_result = tsne.fit_transform(tsne_input)
tsne_result_transpose = rearrange(tsne_result, "x y -> y x")

In [31]:
word_list = tokenizer.batch_decode([token for token in sentence_tokens])

In [33]:
word_dict = {
    'name': word_list + [f"<pos{i}>" for i in range(len(sentence_tokens))], 
    'pca component 1': projected_transpose[0], 
    'pca component 2': projected_transpose[1],
    'tsne component 1': tsne_result_transpose[0],
    'tsne component 2': tsne_result_transpose[1],
    'type': ["token" for _ in sentence_tokens] + ["position" for _ in sentence_tokens]
}
df = pd.DataFrame(data=word_dict)
df

,name,pca component 1,pca component 2,tsne component 1,tsne component 2,type
0,Mr,0.119184,0.028211,-22.170328,21.667768,token
1,.,1.373840,-0.137226,-29.202820,-34.734974,token
2,and,1.369397,-0.174002,-19.339617,-33.723106,token
3,Mrs,0.173536,-0.024907,-24.145966,19.910742,token
4,.,1.373840,-0.137226,-26.392672,-39.450985,token
...,...,...,...,...,...,...
685,<pos340>,-0.284483,1.514739,35.350182,-12.940512,position
686,<pos341>,-0.235062,1.481686,33.724392,-13.847158,position
687,<pos342>,-0.295135,1.441330,35.043026,-14.909719,position
688,<pos343>,-0.262785,1.390339,36.626602,-14.238077,position


In [36]:
x = "tsne component 1"
y = "tsne component 2"
fig = px.scatter(
    df, 
    x=x, 
    y=y, 
    hover_name="name",
    color="type",
    labels={
        x: "Component 1",
        y: "Component 2",
    },
    width=650, 
    height=650, 
)
fig.update_traces(
    marker=dict(size=1),
    selector=dict(mode='markers'),
)
fig.show()

In [ ]:
# !pip install faiss-gpu==1.6.5
# !pip install tsnecuda==3.0.0 --no-deps
# !pip install mkl
# # need to install MKL: https://github.com/eddelbuettel/mkl4deb

In [ ]:
import json
import days.w3d2.w3d2_tests as w3d2_tests
import transformers
import torch
from einops import rearrange, reduce, repeat

from sklearn.decomposition import PCA
from tsnecuda import TSNE
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

In [ ]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
tokenizer._add_tokens(["[BEGIN]", "[END]"])
tokenizer.pad_token = "[END]"
tokenizer.eos_token = "[END]"
# 50258 is the pad_token_id
# 50257 is the BEGIN token id

with open("../w3d2/test_tokens_owt_subset.json") as f:
    test_tokens = torch.LongTensor(json.load(f))[:,:512]

sentence = (
"Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly " +
"normal, thank you very much. They were the last people you’d expect to be involved in anything " +
"strange or mysterious, because they just didn’t hold with such nonsense.\n" +
"Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy " +
"man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin " +
"and blonde and had nearly twice the usual amount of neck, which came in very useful as she " +
"spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys " +
"had a small son called Dudley and in their opinion there was no finer boy anywhere.\n" +
"The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was " +
"that somebody would discover it. They didn’t think they could bear it if anyone found out about " +
"the Potters. Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t met for several years; in fact, " +
"Mrs. Dursley pretended she didn’t have a sister, because her sister and her good-for-nothing " +
"husband were as unDursleyish as it was possible to be. The Dursleys shuddered to think what the " +
"neighbors would say if the Potters arrived in the street. The Dursleys knew that the Potters had a " +
"small son, too, but they had never even seen him. This boy was another good reason for keeping " +
"the Potters away; they didn’t want Dudley mixing with a child like that."
)
sentence_tokens = tokenizer(sentence).input_ids

In [ ]:
# True -> use GPT
# False -> use 2-layer attention-only model
use_gpt = False

if use_gpt:
    model = transformers.GPT2Model.from_pretrained('gpt2')
    VOCAB_SIZE = 50257
    MAX_SEQ_LEN = 1024
    token_embed_fn = model.wte
    pos_embed_fn = model.wpe
else:
    model = w3d2_tests.get_minigpt("../w3d2/model.pt")
    VOCAB_SIZE = 50259
    MAX_SEQ_LEN = 512
    token_embed_fn = model.token_embedding
    pos_embed_fn = model.pos_embedding

In [ ]:
with torch.no_grad():
    all_token_embeddings = token_embed_fn(torch.LongTensor(sentence_tokens))
all_token_embeddings.shape

torch.Size([359, 256])

In [ ]:
with torch.no_grad():
    all_position_embeddings = pos_embed_fn(torch.arange(len(sentence_tokens)))
all_position_embeddings.shape

torch.Size([359, 256])

In [ ]:
X = torch.concat([all_token_embeddings, all_position_embeddings], dim=0)

pca = PCA(n_components=2)
print(all_token_embeddings.shape)
projected = pca.fit_transform(X)
projected_transpose = rearrange(projected, "x y -> y x")

torch.Size([345, 256])


In [ ]:
tsne_input = X
tsne = TSNE()
tsne_result = tsne.fit_transform(tsne_input)
tsne_result_transpose = rearrange(tsne_result, "x y -> y x")

In [ ]:
word_list = tokenizer.batch_decode([token for token in sentence_tokens])

In [ ]:
word_dict = {
    'name': word_list + [f"<pos{i}>" for i in range(len(sentence_tokens))], 
    'pca component 1': projected_transpose[0], 
    'pca component 2': projected_transpose[1],
    'tsne component 1': tsne_result_transpose[0],
    'tsne component 2': tsne_result_transpose[1],
    'type': ["token" for _ in sentence_tokens] + ["position" for _ in sentence_tokens]
}
df = pd.DataFrame(data=word_dict)
df

,name,pca component 1,pca component 2,tsne component 1,tsne component 2,type
0,Mr,0.119184,0.028211,-22.170328,21.667768,token
1,.,1.373840,-0.137226,-29.202820,-34.734974,token
2,and,1.369397,-0.174002,-19.339617,-33.723106,token
3,Mrs,0.173536,-0.024907,-24.145966,19.910742,token
4,.,1.373840,-0.137226,-26.392672,-39.450985,token
...,...,...,...,...,...,...
685,<pos340>,-0.284483,1.514739,35.350182,-12.940512,position
686,<pos341>,-0.235062,1.481686,33.724392,-13.847158,position
687,<pos342>,-0.295135,1.441330,35.043026,-14.909719,position
688,<pos343>,-0.262785,1.390339,36.626602,-14.238077,position


In [ ]:
x = "tsne component 1"
y = "tsne component 2"
fig = px.scatter(
    df, 
    x=x, 
    y=y, 
    hover_name="name",
    color="type",
    labels={
        x: "Component 1",
        y: "Component 2",
    },
    width=650, 
    height=650, 
)
fig.update_traces(
    marker=dict(size=1),
    selector=dict(mode='markers'),
)
fig.show()